In [1]:
import psycopg2
import datetime
import cv2
import urllib
import numpy as np

In [2]:
connection = psycopg2.connect(database='developer', user='postgres', password='4321', host='localhost')

In [3]:
cursor=connection.cursor()

In [4]:
cursor.execute('SELECT * FROM printing')

In [5]:
for row in cursor:
    print(row)

In [6]:
from database import CursorFromConnectionPool
from database import Database

In [6]:
# no partID required for input, generate automatically from database
class Print:
    def __init__(self, status, assigned_printerid, initiate_time, start_time, finish_time, cooldown_time, pickup_time, \
                 cleanup_time, snapshot_path, filamentid, used_filament):
        self.status = status
        self.assigned_printerid = assigned_printerid
        self.start_time = start_time
        self.initiate_time = initiate_time
        self.finish_time = finish_time
        self.cooldown_time = cooldown_time
        self.pickup_time = pickup_time
        self.cleanup_time = cleanup_time
        self.snapshot_path = snapshot_path
        self.filamentid = filamentid
        self.used_filament = used_filament
    
#     def save_to_db(self):
#         with CursorFromConnectionPool() as cursor:
#             cursor.execute('INSERT INTO printing (status, assigned_printerid, start_time, finish_time, filamentid, used_filament) VALUES (%s, %s, %s, %s, %s, %s)',
#                             (self.status, self.assigned_printerid, self.start_time, self.finish_time, self.filamentid, self.used_filament))
    @classmethod
    def load_from_db_by_part(cls, partid):
        with CursorFromConnectionPool() as cursor:
            cursor.execute('SELECT * FROM printing WHERE partid=%s', (partid,))
            print_data = cursor.fetchone()
            return cls(status=print_data[1], assigned_printerid=print_data[2], initiate_time=print_data[3], 
                       start_time=print_data[4], finish_time=print_data[5], cooldown_time=print_data[6],
                       pickup_time=print_data[7], cleanup_time=print_data[8], snapshot_path=print_data[9],
                       filamentid=print_data[10], used_filament=print_data[11],)
    
    @classmethod
    # get current print job on printerX. either waiting or printing. Each printer can have only one job at a time
    def get_current_partid_by_printer(cls, printerid):
        with CursorFromConnectionPool() as cursor:
            cursor.execute('SELECT partid FROM printing WHERE assigned_printerid=%s AND (status=%s OR status=%s\
             OR status=%s OR status=%s)', (printerid, 'waiting','printing', 'cooling', 'onbed',))
            return cursor.fetchone()[0]
    @classmethod
    # operational --> heating
    def initiate_part_to_printerid(cls, printerid): 
        now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with CursorFromConnectionPool() as cursor:
            cursor.execute('INSERT INTO printing (status, assigned_printerid, initiate_time) VALUES (%s, %s, %s)',
                            ('waiting', printerid, now,))
    @classmethod
    # heating --> printing
    def print_part_by_printer(cls, printerid):
        partid = Print.get_current_partid_by_printer(printerid)
        now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with CursorFromConnectionPool() as cursor:
            cursor.execute('UPDATE printing SET start_time=%s WHERE partid=%s;', (now, partid,))
            cursor.execute('UPDATE printing SET status=%s WHERE partid=%s;', ('printing', partid,))
    @classmethod
    # printing --> cooldown
    # cancelling --> cooldown
    # Keenan: Input finished percentage and estimated required filament
    def cooldown_print_by_printer(cls, printerid, percentage, estimated_filament):
        partid = Print.get_current_partid_by_printer(printerid)
        used_filament = float(percentage) * float(estimated_filament)
        now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with CursorFromConnectionPool() as cursor:
            cursor.execute('UPDATE printing SET finish_time=%s WHERE partid=%s;', (now, partid,))
            cursor.execute('UPDATE printing SET status=%s WHERE partid=%s;', ('cooling', partid,))
            cursor.execute('UPDATE printing SET used_filament=%s WHERE partid=%s;', (used_filament, partid,))
    @classmethod
    # printing cancelled from partonbed --> cleanup
    def pickup_cancel_print_by_printer(cls, printerid):
        partid = Print.get_current_partid_by_printer(printerid)
        now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with CursorFromConnectionPool() as cursor:
            cursor.execute('UPDATE printing SET pickup_time=%s WHERE partid=%s;', (now, partid,))
            cursor.execute('UPDATE printing SET status=%s WHERE partid=%s;', ('cancelled', partid,))
    @classmethod
    # printing finished from partonbed --> cleanup
    def pickup_finish_print_by_printer(cls, printerid):
        partid = Print.get_current_partid_by_printer(printerid)
        now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with CursorFromConnectionPool() as cursor:
            cursor.execute('UPDATE printing SET pickup_time=%s WHERE partid=%s;', (now, partid,))
            cursor.execute('UPDATE printing SET status=%s WHERE partid=%s;', ('finished', partid,))
    @classmethod
    # cleaning --> operational
    def cleanedup_by_printer(cls, printerid):
        now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with CursorFromConnectionPool() as cursor:
            cursor.execute('SELECT partid FROM printing WHERE assigned_printerid=%s AND cleanup_time IS NULL ORDER BY pickup_time DESC;', (printerid,))
            partid = cursor.fetchone()[0]
            cursor.execute('UPDATE printing SET cleanup_time=%s WHERE partid=%s;', (now, partid,))
    @classmethod
    def snapshot_by_printer(cls, printerid):
        partid = Print.get_current_partid_by_printer(printerid)
        req = urllib.request.urlopen("http://octopi"+str(printerid)+":8080/?action=snapshot")
        arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
        img = cv2.imdecode(arr, -1)
        return img
    @classmethod
    # cooldown --> partonbed
    # save a snapshot and also save path to SQL database
    def onbed_by_printer(cls, printerid):
        partid = Print.get_current_partid_by_printer(printerid)
        now = datetime.datetime.now()
        img = Print.snapshot_by_printer(printerid)
        path = 'snapshots/Printer'+str(printerid)+'_'+str(now.strftime("%Y-%m-%d_%H-%M-%S"))+'.png'
        cv2.imwrite(path, img)
        with CursorFromConnectionPool() as cursor:
            cursor.execute('UPDATE printing SET cooldown_time=%s WHERE partid=%s;', (now.strftime("%Y-%m-%d %H:%M:%S"), partid,))
            cursor.execute('UPDATE printing SET status=%s WHERE partid=%s;', ('onbed', partid,))
            cursor.execute('UPDATE printing SET snapshot_path=%s WHERE partid=%s;', (path, partid,))
#----------------------under construction from here downwards--------------------------
    
    

In [7]:
# old demo version
class Print:
    def __init__(self, partid, status, assigned_printerid, start_time, finish_time, filamentid, used_filament):
        self.partid = partid
        self.status = status
        self.assigned_printerid = assigned_printerid
        self.start_time = start_time
        self.finish_time = finish_time
        self.filamentid = filamentid
        self.used_filament = used_filament

    def __repr__(self):
        return "<partid {}>".format(self.partid)
    
    def save_to_db(self):
        # This is creating a new connection pool every time! Very expensive...
        with CursorFromConnectionPool() as cursor:
            cursor.execute('INSERT INTO printing (partid, status, assigned_printerid, start_time, finish_time, filamentid, used_filament) VALUES (%s, %s, %s, %s, %s, %s, %s)',
                            (self.partid, self.status, self.assigned_printerid, self.start_time, self.finish_time, self.filamentid, self.used_filament))

    @classmethod
    def load_from_db_by_part(cls, partid):
        with CursorFromConnectionPool() as cursor:
            # Note the (email,) to make it a tuple!
            cursor.execute('SELECT * FROM printing WHERE partid=%s', (partid,))
            print_data = cursor.fetchone()
            return cls(partid=print_data[0], status=print_data[1], part_type=print_data[2], colour=print_data[3],
                       assigned_printerid=print_data[4], start_time=print_data[5], finish_time=print_data[6], material=print_data[7], used_filament=print_data[8])

#----------------------under construction from here downwards--------------------------
    @classmethod
    def update_db_by_part(cls, partid, value):
        with CursorFromConnectionPool() as cursor:
            # Note the (email,) to make it a tuple!
            cursor.execute('UPDATE printing SET used_filament=%s WHERE partid=%s;', (value, partid,))
#             print_data = cursor.fetchone()
#             return cls(partid=print_data[0], status=print_data[1], part_type=print_data[2], colour=print_data[3],
#                        assigned_printerid=print_data[4], start_time=print_data[5], finish_time=print_data[6], material=print_data[7], used_filament=print_data[8])

In [7]:
Database.initialise(database="developer", user="postgres", password="4321", host="localhost")

In [11]:
Print.cooldown_print_by_printer('3',0.5,100.25)

In [56]:
req = urllib.request.urlopen("http://octopi3:8080/?action=snapshot")
arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
img = cv2.imdecode(arr, -1)
now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
print(now)
cv2.imwrite('snapshots/'+str(now)+'.png', img)

2022-09-12_14-30-20


True

In [42]:
datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

'2022-09-12 14:26:02'

In [55]:
now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(now)

AttributeError: 'str' object has no attribute 'type'

In [29]:
user_from_db = Print.load_from_db_by_part(Print.get_printing_partid_by_printer('2'))

print(user_from_db.start_time)

2022-07-12 00:05:02


In [11]:

Print.save_to_db('testing', '3', '','','','')

TypeError: save_to_db() takes 1 positional argument but 6 were given

In [11]:
n = !python OPCUA_test.py
Print.update_db_by_part('2', n[1])

AttributeError: 'NoneType' object has no attribute 'getconn'

In [16]:
p = Print('testing','100', "2022-09-01 00:05:02", None, None, None)

In [17]:
p.save_to_db()

In [61]:
n = !python OPCUA_test.py

In [63]:
n[1]

'5'